<a href="https://colab.research.google.com/github/laurence-lin/Working-files/blob/main/test_txt_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import glob
import os

from tensorflow.keras.applications import ResNet50
import cv2

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt

import time
import gc

import torch
import torchvision



print('Library imported.')


Library imported.


In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# HYPER PARAMETERS SETTING
import pickle

# Calculate width-height ratio to resize image with same scaling
IMG_PATH = glob.glob('/content/drive/MyDrive/binary_data/train/normal/*.png')
MODEL_PATH = '/content/drive/MyDrive/pretrained_txt_overlap_model/resnet34_new.pickle'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Current device: ', device)



Current device:  cuda:0


In [4]:
from PIL import Image
# Load model and dataset

# Load pre-trained model
with open(MODEL_PATH, 'rb') as f:
  pre_model = pickle.load(f)

print('Pre-trained model loaded.')

# Data-transformer
# Define data augmentation and data loader
from torchvision import transforms

# Define data augmentation methods
data_preprocess = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(0.456, 0.225)
    ])



# Preprocess image for PyTorch data architecture
def process_for_model(img):
  '''
  Process image array for PyTorch model
  img: image path string
  return: Tensor image prepared to input to the model
  '''
  img = Image.open(img).convert('RGB')
  img = data_preprocess(img) # remember to do data preprocessing as in the training stage!! This strongly influence the testing performance
  img = img.to(device)
  img = img.view(1, 3, 889, 929)

  return img



Pre-trained model loaded.


In [14]:

def compare_list(y_pred, y_true):
  '''
  Compare prediction and true label of two list
  '''

  result = [1 if (x1 == x2) else 0 for x1, x2 in zip(y_pred, y_true)]
  return result



In [24]:
# Test pre-trained model performance

print('Test pre-trained model performance...')

# Test the model performance

normal_path = glob.glob('/content/drive/MyDrive/binary_data/val/normal/*.png')
overlap_path = glob.glob('/content/drive/MyDrive/binary_data/val/overlap/*.png')

test_path = normal_path + overlap_path
labels = [1 if 'add' in file else 0 for file in test_path]

label_dict = {0:'normal', 1:'overlap'}

print('Start testing prediction...')

predictions = []
for i in range(len(test_path)):
  img = process_for_model(test_path[i]) # data preprocessing

  predict = pre_model(img).cpu().detach().numpy().argmax() # make prediction
  predictions.append(predict)



print('Finish testing prediction.')

compare_result = np.array(compare_list(predictions, labels))
accuracy = compare_result.sum() / len(compare_result)


print('Overall accuracy: ', accuracy)





Test pre-trained model performance...
Start testing prediction...
Finish testing prediction.
Overall accuracy:  1.0
